In [1]:
# Importando bibiotecas
import gurobipy as gb

In [3]:
# Definindo dados

#   Conjuntos: recursos e trabalhos
I = ['Carlos', 'Jonas', 'Monica']
J = ['Cientista', 'Desenvolvedor', 'Arquiteto']

#   Matriz percentual
#   (multidicionárrio)
key, perc = gb.multidict({
    ('Carlos', 'Cientista'): 53,
    ('Carlos', 'Desenvolvedor'): 27,
    ('Carlos', 'Arquiteto'): 13,
    ('Jonas', 'Cientista'): 80,
    ('Jonas', 'Desenvolvedor'): 47,
    ('Jonas', 'Arquiteto'): 67,
    ('Monica', 'Cientista'): 53,
    ('Monica', 'Desenvolvedor'): 73,
    ('Monica', 'Arquiteto'): 47,
})

In [15]:
# Definindo Modelo
m = gb.Model("Atribuicao")

# Adicionando variaveis
x = m.addVars(key, vtype=gb.GRB.BINARY, name='x')
m.update()
# Adicionando Restrições

#   Trabalho
trabalho = m.addConstrs((x.sum('*', j) == 1 for j in J), 'trabalho')
m.update()
#   Pessoa
pessoa = m.addConstrs((x.sum(i, '*') <= 1 for i in I), 'pessoa')
m.update()
# Definindo a função objetivo
m.setObjective(x.prod(perc), gb.GRB.MAXIMIZE)

m.update()

In [17]:
m.write('atribuicao.lp')

In [19]:
# Resolvendo o problema
m.optimize()

# Imprimindo as soluções
m.printAttr('x')

# Imprimindo o valor da função Objetivo
m.objVal

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x8e59071a
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 6 rows, 9 columns, 18 nonzeros

Continuing optimization...


Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 193 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.930000000000e+02, best bound 1.930000000000e+02, gap 0.0000%

    Variable            x 
-------------------------
x[Carlos,Cientista]            1 
x[Jonas,Arquiteto]            1 
x[Monica,Desenvolvedor]            1 


193.0

In [21]:
# Imprimindo solucao
for var in m.getVars():
    if abs(var.x) > 1e-6:
        print(var.varName, '=', var.x)

x[Carlos,Cientista] = 1.0
x[Jonas,Arquiteto] = 1.0
x[Monica,Desenvolvedor] = 1.0


In [34]:
# Adicionando um novo canditado (Novo  cenário). 
nperc = {('Ana', 'Cientista'): 100,
         ('Ana', 'Desenvolvedor'): 100,
         ('Ana', 'Arquiteto'): 100}

for key, val in nperc.items():
    i, j = key
    x[key] = m.addVar(obj=val, 
                      name='x[{0}, {1}]'.format(i, j),
                      column=gb.Column([1], [m.getConstrByName('trabalho[{0}]'.format(j))])
                      )
m.addConstr(x.sum('Ana', '*') <= 1, name='pessoa[Ana]')

<gurobi.Constr *Awaiting Model Update*>

In [35]:
m.update()

In [37]:
# Resolvendo o problema
m.optimize()

# Imprimindo as soluções
m.printAttr('x')

# Imprimindo o valor da função Objetivo
m.objVal

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x2fd76783
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 7 rows, 12 columns, 24 nonzeros

Continuing optimization...


Explored 0 nodes (4 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 253 193 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.530000000000e+02, best bound 2.530000000000e+02, gap 0.0000%

    Variable            x 
-------------------------
x[Jonas,Cientista]            1 
x[Monica,Desenvolvedor]            1 
x[Ana, Arquiteto]            1 


253.0

In [38]:
m.write('atribuicao_2.lp')

In [ ]:
#Começanco Algoritmo 1 (Dantzig relaxado)